### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# number of movies
print("The number of movies is {}.".format(movies.shape[0]))

# number of ratings
print("The number of ratings is {}.".format(reviews.shape[0]))

# unique users
print("The number of unique users is {}.".format(reviews.user_id.nunique()))

# missing ratings
print("The number of missing reviews is {}.".format(int(reviews.rating.isnull().mean()*reviews.shape[0])))

# the average, min, and max ratings given
print("The average, minimum, and max ratings given are {}, {}, and {}, respectively.".format(np.round(reviews.rating.mean(), 0), reviews.rating.min(), reviews.rating.max()))

The number of movies is 35479.
The number of ratings is 863866.
The number of unique users is 67353.
The number of missing reviews is 0.
The average, minimum, and max ratings given are 7.0, 0, and 10, respectively.


In [3]:
movies

movie_id                                              movie  \
0       0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1       0000010                La sortie des usines Lumière (1895)   
2       0000012                      The Arrival of a Train (1896)   
3            25  The Oxford and Cambridge University Boat Race ...   
4       0000091                         Le manoir du diable (1896)   
...         ...                                                ...   
35474  12036052                                    Elephant (2020)   
35475  12058882           Coachella: 20 Years in the Desert (2020)   
35476  12078990                 Money Heist: The Phenomenon (2020)   
35477  12131262                              Out of Shadows (2020)   
35478  12192654                        Planet of the Humans (2019)   

                                        genre  
0                           Documentary|Short  
1                           Documentary|Short  
2                           Documentary|Short  
3                                         NaN  
4                                Short|Horror  
...                                       ...  
35474  Documentary|Adventure|Biography|Family  
35475                       Documentary|Music  
35476                             Documentary  
35477                             Documentary  
35478                                     NaN  

[35479 rows x 3 columns]

In [4]:
reviews

user_id  movie_id  rating   timestamp
0            1   0114508       8  1381006850
1            2   0208092       5  1586466072
2            2   0358273       9  1579057827
3            2  10039344       5  1578603053
4            2   6751668       9  1578955697
...        ...       ...     ...         ...
863861   67351   9071322       5  1580517448
863862   67351   9898858       3  1585958452
863863   67352   0172495      10  1587107015
863864   67352   0414387      10  1587107852
863865   67353   1623205       6  1362832655

[863866 rows x 4 columns]

In [6]:
# number of different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

The number of genres is 28.


In [13]:
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [7]:
# Use your findings to match each variable to the correct statement in the dictionary
a = reviews.user_id.nunique()
b = reviews.rating.max()
c = np.round(reviews.rating.mean(), 0)
d = movies.shape[0]
e = reviews.rating.min()
f = int(reviews.rating.isnull().mean()*reviews.shape[0])
h = reviews.shape[0]
i = len(genres)

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': e,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [10]:
# pull date if it exists
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# apply the function to pull the date
movies['date'] = movies['movie'].apply(create_date)

# Return century of movie as a dummy column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)


In [16]:
movies

movie_id                                              movie  \
0       0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1       0000010                La sortie des usines Lumière (1895)   
2       0000012                      The Arrival of a Train (1896)   
3            25  The Oxford and Cambridge University Boat Race ...   
4       0000091                         Le manoir du diable (1896)   
...         ...                                                ...   
35474  12036052                                    Elephant (2020)   
35475  12058882           Coachella: 20 Years in the Desert (2020)   
35476  12078990                 Money Heist: The Phenomenon (2020)   
35477  12131262                              Out of Shadows (2020)   
35478  12192654                        Planet of the Humans (2019)   

                                        genre  date  1800's  1900's  2000's  
0                           Documentary|Short  1894       1       0       0  
1                           Documentary|Short  1895       1       0       0  
2                           Documentary|Short  1896       1       0       0  
3                                         NaN  1895       1       0       0  
4                                Short|Horror  1896       1       0       0  
...                                       ...   ...     ...     ...     ...  
35474  Documentary|Adventure|Biography|Family  2020       0       0       1  
35475                       Documentary|Music  2020       0       0       1  
35476                             Documentary  2020       0       0       1  
35477                             Documentary  2020       0       0       1  
35478                                     NaN  2019       0       0       1  

[35479 rows x 7 columns]

In [17]:
# Function to split and return values for columns
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [18]:
movies.head() #Check what it looks like

movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Mystery  War  History  \
0  Documentary|Short  1894       1       0       0        0    0        0   
1  Documentary|Short  1895       1       0       0        0    0        0   
2  Documentary|Short  1896       1       0       0        0    0        0   
3                NaN  1895       1       0       0        0    0        0   
4       Short|Horror  1896       1       0       0        0    0        0   

   ...  Crime  Fantasy  Biography  Musical  Thriller  Film-Noir  Animation  \
0  ...      0        0          0        0         0          0          0   
1  ...      0        0          0        0         0          0          0   
2  ...      0        0          0        0         0          0          0   
3  ...      0        0          0        0         0          0          0   
4  ...      0        0          0        0         0          0          0   

   Romance  Sport  Drama  
0        0      0      0  
1        0      0      0  
2        0      0      0  
3        0      0      0  
4        0      0      0  

[5 rows x 35 columns]

In [19]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [20]:
reviews

user_id  movie_id  rating   timestamp                 date
0            1   0114508       8  1381006850  2013-10-05 23:00:50
1            2   0208092       5  1586466072  2020-04-09 23:01:12
2            2   0358273       9  1579057827  2020-01-15 04:10:27
3            2  10039344       5  1578603053  2020-01-09 21:50:53
4            2   6751668       9  1578955697  2020-01-13 23:48:17
...        ...       ...     ...         ...                  ...
863861   67351   9071322       5  1580517448  2020-02-01 01:37:28
863862   67351   9898858       3  1585958452  2020-04-04 02:00:52
863863   67352   0172495      10  1587107015  2020-04-17 09:03:35
863864   67352   0414387      10  1587107852  2020-04-17 09:17:32
863865   67353   1623205       6  1362832655  2013-03-09 13:37:35

[863866 rows x 5 columns]

In [21]:
# now reviews and movies are the final dataframes with the necessary columns
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')